# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
#import current weather data from csv
clean_weather_data = pd.read_csv('../Output/clean_weather_data.csv')

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = clean_weather_data[["Latitude", "Longitude"]]
humidity = clean_weather_data['Humidity (%)']

fig1 = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 3)

fig1.add_layer(heat_layer)

fig1

Figure(layout=FigureLayout(height='420px'))

### Narrowing Data to My Ideal Weather Conditions
My ideal weather conditions include:
- Anywhere between 75 and 80 degrees.
- A pleasent breeze (5 - 24 mph)
- Some clouds in the sky to admire (20% to 50%)

In [10]:
# narrowing the data to my ideal weather conditions
ideal_weather_data = clean_weather_data.loc[(clean_weather_data['Temperature (F)'] >= 75) & (clean_weather_data['Temperature (F)'] < 80),:]
ideal_weather_data = ideal_weather_data.loc[(ideal_weather_data['Wind Speed (mph)'] >= 5) & (ideal_weather_data['Wind Speed (mph)'] < 24),:]
ideal_weather_data = ideal_weather_data.loc[(ideal_weather_data['Cloudiness (%)'] >= 20) & (ideal_weather_data['Cloudiness (%)'] < 50),:]

ideal_weather_data = ideal_weather_data.set_index('City')
#ideal_weather_data

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
hotel_df = pd.DataFrame(columns = ['Hotel Name'])

In [22]:
test_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=29.4241,-98.4936&radius=5000&type=hotel&key={g_key}'
test_response = requests.get(query_url).json()
test_results = test_response['results']

#test_results

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': -6.886246, 'lng': 112.1966878},
   'viewport': {'northeast': {'lat': -6.884854319708498,
     'lng': 112.1980449302915},
    'southwest': {'lat': -6.887552280291502, 'lng': 112.1953469697085}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
  'icon_background_color': '#7B9EB0',
  'icon_mask_base_uri': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_pinlet',
  'name': 'Musholla Babut Taubat',
  'photos': [{'height': 2592,
    'html_attributions': ['<a href="https://maps.google.com/maps/contrib/104425847892433233049">Barry Raja</a>'],
    'photo_reference': 'Aap_uEC39exWhvW40SC3Zl0fdjrZ8u912Be1IZhjCnpIoxd8aFNb7fCxw62lep2q0bOxgYRCUAhroD2v8YmL3g7uFuFflr9Xjr0inxGH1QBVbB64D5ZMjGm7qs8yHPV0c7giOqKtmfOlTrN2-TiXDz1pNClBYshKFKaNRfAWmx8UPBIaHULV',
    'width': 1456}],
  'place_id': 'ChIJG9CLX9G_dy4RHTzynnYyB20',
  'plus_code': {'compound_code': '457W+GM Lab

In [20]:
# find the closest restaurant of each type to coordinates

search_radius= 5000
search_type = 'hotel'

base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json?'
query_perams = f'&radius={search_radius}&type={search_type}&key={g_key}'

# use iterrows to iterate through pandas dataframe
for index, row in ideal_weather_data.iterrows():
  
    lat = row['Latitude']
    lon = row['Longitude']
        
    # get location type from df
    location = f'location={lat},{lon}'
    
    # assemble url and make API request
    print(f"Searching for Hotel in {index}.")
    query_url = base_url + location + query_perams
    response = requests.get(query_url).json()   
    
    # extract results
    results = response['results']
    
    
    try:
        print(f"Hotel Found in {results[0]['name']}.")  
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        #types_df.loc[index, 'address'] = results[0]['vicinity']
        #types_df.loc[index, 'price_level'] = results[0]['price_level']
        #types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Hotel in atuona.
Hotal in restaurant is Atuona.
------------
Retrieving Hotel in albany.
Hotal in restaurant is Voorheesville.
------------
Retrieving Hotel in faanui.
Hotal in restaurant is Vaitape.
------------
Retrieving Hotel in itarema.
Hotal in restaurant is Itarema.
------------
Retrieving Hotel in touros.
Hotal in restaurant is Touros.
------------
Retrieving Hotel in teahupoo.
Hotal in restaurant is Teahupo'o.
------------
Retrieving Hotel in denpasar.
Hotal in restaurant is Denpasar.
------------
Retrieving Hotel in ambur.
Hotal in restaurant is Ambur.
------------
Retrieving Hotel in harper.
Hotal in restaurant is Harper.
------------
Retrieving Hotel in ushtobe.
Hotal in restaurant is Ushtobe.
------------
Retrieving Hotel in labuhan.
Hotal in restaurant is Musholla Babut Taubat.
------------


In [8]:
 
    try:
        print(f"Closest {restr_type} restaurant is {results[0]['name']}.")
        
        types_df.loc[index, 'name'] = results[0]['name']
        types_df.loc[index, 'address'] = results[0]['vicinity']
        types_df.loc[index, 'price_level'] = results[0]['price_level']
        types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

NameError: name 'restr_type' is not defined

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
